In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv("../Data/ecommerce_furniture_dataset_2024.csv")

In [3]:
dataset.head()

,productTitle,originalPrice,price,sold,tagText
0,Dresser For Bedroom With 9 Fabric Drawers Ward...,NaN,$46.79,600,Free shipping
1,Outdoor Conversation Set 4 Pieces Patio Furnit...,NaN,$169.72,0,Free shipping
2,Desser For Bedroom With 7 Fabric Drawers Organ...,$78.4,$39.46,7,Free shipping
3,"Modern Accent Boucle Chair,Upholstered Tufted ...",NaN,$111.99,0,Free shipping
4,Small Unit Simple Computer Desk Household Wood...,$48.82,$21.37,1,Free shipping


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   productTitle   2000 non-null   object
 1   originalPrice  487 non-null    object
 2   price          2000 non-null   object
 3   sold           2000 non-null   int64 
 4   tagText        1997 non-null   object
dtypes: int64(1), object(4)
memory usage: 78.3+ KB


In [35]:
dataset.describe()

,title_length,originalPrice,price,sold,has_free_shipping,discount_pct
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,116.473500,326.891412,156.560020,23.493500,0.941500,0.526650
std,15.633218,366.787603,176.936735,254.094061,0.234745,0.107007
min,11.000000,3.630000,0.990000,0.000000,0.000000,0.000000
25%,113.000000,108.958262,48.530000,1.000000,1.000000,0.510000
50%,122.000000,243.185750,114.080000,3.000000,1.000000,0.520000
75%,126.000000,398.450470,193.490000,9.000000,1.000000,0.550000
max,128.000000,5685.588037,2876.380000,10000.000000,1.000000,0.950000


In [5]:
# # basic cleaning
dataset['price'] = dataset['price'].str.replace('$', '', regex=False) \
                         .str.replace(',', '', regex=False) \
                         .astype(float)

In [6]:
dataset['originalPrice'].isnull().sum()

1513

In [7]:
dataset['price'].isnull().sum()

0

In [8]:
dataset['tagText'].isnull().sum()

3

In [9]:
dataset.drop_duplicates()

,productTitle,originalPrice,price,sold,tagText
0,Dresser For Bedroom With 9 Fabric Drawers Ward...,NaN,46.79,600,Free shipping
1,Outdoor Conversation Set 4 Pieces Patio Furnit...,NaN,169.72,0,Free shipping
2,Desser For Bedroom With 7 Fabric Drawers Organ...,$78.4,39.46,7,Free shipping
3,"Modern Accent Boucle Chair,Upholstered Tufted ...",NaN,111.99,0,Free shipping
4,Small Unit Simple Computer Desk Household Wood...,$48.82,21.37,1,Free shipping
...,...,...,...,...,...
1992,Oversized Outdoor Swivel Rocker Chairs Set wit...,NaN,555.46,0,Free shipping
1993,6pcs Patio Furniture Set PE Rattan Wicker Sect...,NaN,325.83,4,Free shipping
1994,Garden Furniture 4507 (Dark Grey) Steel Frame ...,NaN,105.16,2,Free shipping
1998,Furniture Acrylic Coffee Table Transparent Liv...,NaN,228.18,0,Free shipping


In [10]:
dataset['tagText'] = dataset['tagText'].fillna(dataset['tagText'].mode()[0])

In [11]:
dataset['tagText'].isnull().sum()

0

In [12]:
# # Extract features from text
# # creating new column to check if customer had free shipping or not

dataset['has_free_shipping'] = dataset['tagText'].str.contains('Free shipping', na=False).astype(int)

In [13]:
dataset.head()

,productTitle,originalPrice,price,sold,tagText,has_free_shipping
0,Dresser For Bedroom With 9 Fabric Drawers Ward...,NaN,46.79,600,Free shipping,1
1,Outdoor Conversation Set 4 Pieces Patio Furnit...,NaN,169.72,0,Free shipping,1
2,Desser For Bedroom With 7 Fabric Drawers Organ...,$78.4,39.46,7,Free shipping,1
3,"Modern Accent Boucle Chair,Upholstered Tufted ...",NaN,111.99,0,Free shipping,1
4,Small Unit Simple Computer Desk Household Wood...,$48.82,21.37,1,Free shipping,1


In [14]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   productTitle       2000 non-null   object 
 1   originalPrice      487 non-null    object 
 2   price              2000 non-null   float64
 3   sold               2000 non-null   int64  
 4   tagText            2000 non-null   object 
 5   has_free_shipping  2000 non-null   int32  
dtypes: float64(1), int32(1), int64(1), object(3)
memory usage: 86.1+ KB


In [15]:
# # title_length: Measures the length of the product title (longer titles may impact sales).

dataset['title_length'] = dataset['productTitle'].str.len()

In [16]:
# # changing data type of column for better calculation

dataset['originalPrice'] = dataset['originalPrice'].str.replace('$', '', regex=False) \
                         .str.replace(',', '', regex=False) \
                         .astype(float)

In [17]:
# # changing column index for better understanding and reading data 

col = dataset.pop('title_length')  # Remove the column
dataset.insert(1, 'title_length', col) 

In [18]:
# # Price bins (categorizing prices into ranges)

dataset['price_bin'] = pd.cut(dataset['price'], bins=[0, 50, 100, 200, 500, np.inf], 
                         labels=['<50', '50-100', '100-200', '200-500', '500+'])

In [19]:
dataset.head()

,productTitle,title_length,originalPrice,price,sold,tagText,has_free_shipping,price_bin
0,Dresser For Bedroom With 9 Fabric Drawers Ward...,127,NaN,46.79,600,Free shipping,1,<50
1,Outdoor Conversation Set 4 Pieces Patio Furnit...,127,NaN,169.72,0,Free shipping,1,100-200
2,Desser For Bedroom With 7 Fabric Drawers Organ...,124,78.40,39.46,7,Free shipping,1,<50
3,"Modern Accent Boucle Chair,Upholstered Tufted ...",128,NaN,111.99,0,Free shipping,1,100-200
4,Small Unit Simple Computer Desk Household Wood...,121,48.82,21.37,1,Free shipping,1,<50


In [24]:
correlation = dataset[['originalPrice', 'price']].corr().iloc[0, 1]
print(f"Correlation between originalPrice and price: {correlation:.2f}")

Correlation between originalPrice and price: 0.84


In [25]:
# So correlation is strong (>0.8), impute using linear regression

from sklearn.linear_model import LinearRegression

In [26]:
# Train on non-null rows

train_df = dataset.dropna(subset=['originalPrice'])

In [27]:
model = LinearRegression()

In [28]:
model.fit(train_df[['price']], train_df['originalPrice'])

LinearRegression()

In [30]:
# Predict missing values
null_mask = dataset['originalPrice'].isna()
dataset.loc[null_mask, 'originalPrice'] = model.predict(dataset.loc[null_mask, ['price']])

In [31]:
dataset.head()

,productTitle,title_length,originalPrice,price,sold,tagText,has_free_shipping,price_bin
0,Dresser For Bedroom With 9 Fabric Drawers Ward...,127,110.618189,46.79,600,Free shipping,1,<50
1,Outdoor Conversation Set 4 Pieces Patio Furnit...,127,352.819724,169.72,0,Free shipping,1,100-200
2,Desser For Bedroom With 7 Fabric Drawers Organ...,124,78.400000,39.46,7,Free shipping,1,<50
3,"Modern Accent Boucle Chair,Upholstered Tufted ...",128,239.077801,111.99,0,Free shipping,1,100-200
4,Small Unit Simple Computer Desk Household Wood...,121,48.820000,21.37,1,Free shipping,1,<50


In [32]:
# # after predicting originalPrice values we can find discount

dataset['discount_pct'] = ((dataset['originalPrice'] - dataset['price']) / dataset['originalPrice']).round(2)

In [33]:
dataset.head()

,productTitle,title_length,originalPrice,price,sold,tagText,has_free_shipping,price_bin,discount_pct
0,Dresser For Bedroom With 9 Fabric Drawers Ward...,127,110.618189,46.79,600,Free shipping,1,<50,0.58
1,Outdoor Conversation Set 4 Pieces Patio Furnit...,127,352.819724,169.72,0,Free shipping,1,100-200,0.52
2,Desser For Bedroom With 7 Fabric Drawers Organ...,124,78.400000,39.46,7,Free shipping,1,<50,0.50
3,"Modern Accent Boucle Chair,Upholstered Tufted ...",128,239.077801,111.99,0,Free shipping,1,100-200,0.53
4,Small Unit Simple Computer Desk Household Wood...,121,48.820000,21.37,1,Free shipping,1,<50,0.56


In [34]:
dataset['discount_pct'].isnull().sum()

0

In [36]:
# # saving cleaned data

dataset.to_csv('../Data/cleaned_data.csv', index=False)